In [7]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import pickle
import numpy as np
from dataset.mnist import load_mnist
from PIL import Image

# 시그모이드 함수 ( 1 / (1+exp(-x)) )
def sigmoid(x) :
    return 1 / (1 + np.exp(-x))

# 소프트맥스 함수(입력값 각각의 정답일 확률 계산시에 활성화 함수로 사용)
def softmax(a) :
    c = np.max(a) # 오버플로 방지용, 소프트맥스 함수의 지수 함수를 계산할 때 어떤 정수를 더해도 결과는 안 바뀌니 입력 신호 중 최댓값을 이용
    exp_a = np.exp(a-c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()
    
# 여기선 학습용 데이터가 아닌 검증용 데이터만 가져옴
def get_data():
    """
    normalize를 True로 하면 데이터들이
    전처리(신경망의 입력 데이터에 특정 변환을 가하는 것) 작업으로
    정규화(특정 범위(여기선 0.0 ~ 1.0)로 변환하는 처리)를 수행한다.
    
    Tip.
    전처리를 통해 식별 능력을 개선하고 학습 속도를 높이는 등의 사례가 많이 있고
    정규화 외에도 백색화(전체 데이터를 균일하게 분포시키는 것) 등 다양한 전처리가 있다.
    """
    (x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)
    return x_test, t_test

def init_network():
    with open("sample_weight.pkl", "rb") as f:
        network = pickle.load(f)
        
    return network

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x, W1)+b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2)+b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3)+b3
    y = softmax(a3)
    
    return y

x, t= get_data()
network = init_network()

batch_size = 100 # 배치(여러 입력 데이터를 한 묶음으로 묶은 것)
accuracy_cnt = 0
for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1) # axis=n, 첫 열부터 n 만큼 묶어서 계산(어떤 열이 가장 정답에 근접한 값들을 가지고 있나)하여 인덱스 추출
    accuracy_cnt += np.sum(p == t[i:i+batch_size])
    
"""
3.6.2 신경망의 추론처리 내용
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y) # 확률이 가장 높은 원소의 인덱스 얻는다.
    if p == t[i]: # 추론값(입력값들 중 가장 정답일거라 제시한 값)과 정답값이 같을 시에
        accuracy_cnt += 1 # 정확성 카운트 +1
"""
        
print("정확도 :", float(accuracy_cnt)/len(x))

"""
3.6.1 mnist 데이터셋 내용

img = x_train[0]
label = t_train[0]
print(label)  # 5

print(img.shape)  # (784,)
img = img.reshape(28, 28)  # 형상을 원래 이미지의 크기로 변형 (= 원하는 형상을 인수로 지정하면 넘파이 배열의 형상을 바꿀 수 있다.)
print(img.shape)  # (28, 28)

img_show(img) # 이미지 출력
"""

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in exp
  # This is added back by InteractiveShellApp.init_path()


정확도 : 0.9207


'\n3.6.1 mnist 데이터셋 내용\n\nimg = x_train[0]\nlabel = t_train[0]\nprint(label)  # 5\n\nprint(img.shape)  # (784,)\nimg = img.reshape(28, 28)  # 형상을 원래 이미지의 크기로 변형 (= 원하는 형상을 인수로 지정하면 넘파이 배열의 형상을 바꿀 수 있다.)\nprint(img.shape)  # (28, 28)\n\nimg_show(img) # 이미지 출력\n'